In [1]:

import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout, Masking
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.advanced_activations import PReLU
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

import matplotlib.pyplot as plt

from normalize import normalize

import plotly.plotly as py
from plotly.graph_objs import *

Using Theano backend.


In [2]:
loadHis = pd.read_csv('final.csv')

In [3]:
for i in range(0, loadHis.shape[0] - 1):
    if((loadHis[i:i+1].isnull().sum()['h1'] < 1) and ((loadHis[i:i+1].isnull().sum()['h24'] == 1))):
        info = [loadHis.loc[i, 'zone_id'], loadHis.loc[i, 'year'], loadHis.loc[i, 'month'], loadHis.loc[i, 'day']]
        loadHis.loc[i] = loadHis.loc[i-7]
        loadHis.loc[i, 'zone_id'] = info[0]
        loadHis.loc[i, 'year'] = info[1]
        loadHis.loc[i, 'month'] = info[2]
        loadHis.loc[i, 'day'] = info[3]

In [3]:
loadHis.dropna()
loadHis.to_csv('final.csv', index=None)

In [4]:
res = loadHis.sort_values([u'year', u'month', u'day', u'zone_id'], ascending=[1, 1, 1, 1])
trainRes = res[[u'h1', u'h2', u'h3', u'h4',
       u'h5', u'h6', u'h7', u'h8', u'h9', u'h10', u'h11', u'h12', u'h13',
       u'h14', u'h15', u'h16', u'h17', u'h18', u'h19', u'h20', u'h21', u'h22',
       u'h23', u'h24']]

In [5]:
res = trainRes.dropna()

In [6]:
res.shape

(31740, 24)

In [7]:
batch_size1 = 1587
tsteps1 = res.shape[0] * 24 / batch_size1 / 20

#trainBuffer = res.as_matrix().T
rawData = res.as_matrix().T
reshapedData = np.reshape(rawData, (batch_size1, tsteps1, 20))
reshapedData.shape

(1587, 24, 20)

In [8]:
usedData = reshapedData[0:1582, :, :]
batch_size1 = 226
tsteps1 = usedData.shape[0] / batch_size1 * 24

usedData = np.reshape(usedData, (batch_size1, tsteps1, 20))
usedData.shape

(226, 168, 20)

In [9]:
train = usedData[0:224, :, :]
test = usedData[224:225, :, :]


train_Y = usedData[1:225, :, :]
test_Y = usedData[225:226, :, :]


In [10]:
train_Y = train

In [11]:
tsteps1

168

In [12]:
ts = Sequential()
ts.add(Masking(mask_value=0., input_shape=(tsteps1, 20)))
ts.add(TimeDistributed(Dense(output_dim=200), input_shape=(tsteps1, 20)))
ts.add(PReLU())
#ts.add(Dropout(0.4))
#ts.add(LSTM(100, batch_input_shape=(60, 24, 20), activation='relu', 
#            inner_activation='relu', return_sequences=True, stateful=True))
ts.add(LSTM(output_dim=500,
           input_shape=(tsteps1, 200),
           dropout_W = 0.4,
           return_sequences=True))
ts.add(LSTM(output_dim=100,
           input_shape=(tsteps1, 500),
           dropout_W = 0.4,
           return_sequences=True))
#ts.add(Dropout(0.4))
ts.add(TimeDistributed(Dense(output_dim=1), input_shape=(tsteps1, 100)))
ts.add(PReLU())

ts.compile(loss='mse', optimizer='rmsprop')

In [13]:
train_norm = normalize(train)
train_norm.normedData[0, 2, :]

array([-0.8988923 ,  0.63161272,  0.76761441, -1.08423014, -1.01511193,
        0.70632439,  0.76761441, -1.05224297, -0.0515429 , -0.78901985,
       -0.01759867,  0.29449778, -0.87039607, -0.84937102, -0.3197512 ,
       -0.76501955, -0.71448008,  1.07837531, -0.19784555, -0.18279742])

In [14]:
trainLabel_norm = normalize(train_Y)
trainLabel_norm.normedData[0, 1, :]

array([-0.90264771,  0.54261988,  0.6716    , -1.08372766, -1.01176643,
        0.62069027,  0.6716    , -1.05011401, -0.0776457 , -0.81789955,
        0.05879236,  0.41796378, -0.85052145, -0.82839886, -0.33553982,
       -0.74845075, -0.72939597,  1.22627798, -0.2103019 , -0.14356388])

In [15]:
epochs1 = 40

ts.fit(train_norm.normedData,
        trainLabel_norm.normedData[:, :, 1:2],
          verbose=1,
          nb_epoch=epochs1)

/Users/king/anaconda/lib/python2.7/site-packages/Keras-1.0.0-py2.7.egg/keras/backend/theano_backend.py:484: UserWarning:

theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 3 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.



Epoch 1/40
224/224 [==============================] - 12s - loss: 1.7165    
Epoch 2/40
224/224 [==============================] - 12s - loss: 1.5922    
Epoch 3/40
224/224 [==============================] - 12s - loss: 1.3624    
Epoch 4/40
224/224 [==============================] - 13s - loss: 1.0599    
Epoch 5/40
224/224 [==============================] - 13s - loss: 0.7670    
Epoch 6/40
224/224 [==============================] - 13s - loss: 0.5595    
Epoch 7/40
224/224 [==============================] - 13s - loss: 0.4306    
Epoch 8/40
224/224 [==============================] - 11s - loss: 0.3457    
Epoch 9/40
224/224 [==============================] - 13s - loss: 0.2874    
Epoch 10/40
224/224 [==============================] - 13s - loss: 0.2515    
Epoch 11/40
224/224 [==============================] - 13s - loss: 0.2308    
Epoch 12/40
224/224 [==============================] - 13s - loss: 0.2173    
Epoch 13/40
224/224 [==============================] - 13s - loss: 0.1979

In [16]:
y = ts.predict(train_norm.normedData, batch_size=batch_size1)

/Users/king/anaconda/lib/python2.7/site-packages/Keras-1.0.0-py2.7.egg/keras/backend/theano_backend.py:484: UserWarning:

theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.



In [51]:
trainLabel_norm.normedData[0, :, 0]

array([-0.90280165, -0.89904355, -0.94093847, -0.9478592 , -0.88180126,
       -0.74587434, -0.75791615, -0.80990767, -0.78039861, -0.68778252,
       -0.78659154, -0.861661  , -0.85479319, -0.86331509, -0.78997913,
       -0.79512667, -0.85925263, -0.8354469 , -0.73916533, -0.75556072,
       -0.77081809, -0.76880671, -0.81504198, -0.79807758, -0.69417394,
       -0.76193891, -0.76926986, -0.77666697, -0.82947892, -0.71911769,
       -0.75275537, -0.78324365, -0.82876435, -0.84240733, -0.81115155,
       -0.80023452, -0.83503668, -0.78813978, -0.80330452, -0.83474556,
       -0.85286121, -0.84640362, -0.810609  , -0.86417522, -0.8516438 ,
       -0.79950672, -0.76557792, -0.77338525, -0.84959272, -0.88259522,
       -0.92291545, -0.85652668, -0.8423544 , -0.86155513, -0.87024906,
       -0.83096099, -0.83683634, -0.82799685, -0.86659682, -0.9177282 ,
       -0.9370745 , -0.93000821, -0.95286119, -0.95757205, -0.95284795,
       -0.94877226, -0.92683234, -0.86585578, -0.86565729, -0.82

In [20]:
yprint0 = y[4, :, 0:1].flatten()
yprint1 = trainLabel_norm.normedData[4,:, 1:2].flatten()

plt.plot(yprint0, 'b', yprint1, 'g')
#plt.plot(yprint0, 'b')
#plt.plot(yprint1, 'r')
plt.show()

In [ ]:
#yprint0 = y[50, :, 11:12].flatten()
yprint1 = trainLabel_norm.normedData[57, :, 11:12].flatten()
trainprint1 = train_norm.normedData[50, :, 11:12].flatten()

plt.plot(yprint1, 'r', trainprint1, 'g')
plt.show()

In [57]:
trace0 = Scatter(
  y= np.reshape(trainLabel_norm.normedData[:,:, 0:1], (1580 * 24))
)
trace1 = Scatter(
  y=np.reshape(y[:, :, 0:1], (1580 * 24))
)
data = Data([trace0, trace1])

py.iplot(data, filename = 'basic-line')

The draw time for this plot will be slow for all clients.


In [ ]:
test1 =  train_norm.cal(test)
test_Y1 = trainLabel_norm.cal(test_Y)


y = ts.predict(test1)

ypred = y[:,:, 11:12].flatten()
ystandard = test_Y1[:,:, 11:12].flatten()


plt.plot(ypred, 'b', ystandard, 'g')
plt.show()

test1 = np.zeros(test.shape)
test1.astype(float)

test1[:, 0:24*6, :] =  train_norm.cal(test[:, 0:24*6, :])


for i in range(24*6 + 1, 240):
    y = ts.predict(test1)
    test1[:, i, :] = y[:, i - 1, :]
    

In [21]:
y[:, :, 0:1].shape

(1, 240, 1)

In [22]:
plt.plot(y[:, :, 11:12].flatten(), 'r', test_Y1[:,:, 11:12].flatten(), 'b')
plt.show()

In [40]:
ypre = trainLabel_norm.recover(y).flatten()
ypre

array([  1.19342692e+09,   1.20450108e+10,   1.29966308e+10, ...,
         1.00515402e+10,   3.82577328e+09,   4.72301281e+09])

In [41]:
y = train_Y.flatten()

In [43]:
((y - ypre) ** 2).mean()

7.160885216032238e+19

In [59]:
plt.plot(y, 'r', ypre, 'b')
plt.show()

KeyboardInterrupt: 